<a href="REF: https://github.com/GoogleCloudPlatform/df-ml-anomaly-detection/blob/master/README.md">Reference1<a/>  <br>
<a href="https://faker.readthedocs.io/en/master/">Reference2<a/> 

In [80]:
# %%writefile datagenerator.py
# import numpy as np

from faker import Faker
# from dataclasses import dataclass
# from datetime import datetime, timedelta
# import geocoder
# from random import choice

In [15]:
# export  NUM_DEST_IP=20
# export  NUSERS=500
# export  AVG_REQUESTS_SECS=10
# export  MAX_REQUEST_BYTES=5000
# export  MIN_SESSION_EVENTS=5
# export  MAX_SESSION_EVENTS=20
# export  PROJECT_ID="electric-armor-395015"
# export  TOPIC_NAME="netlog-stream-topic"

In [ ]:
# Feature Extraction After Aggregation
# Added processing timestamp.
# Group by destination subnet and subscriberId
# Number of approximate unique src IP
# Number of approximate unique src port
# Number of unique records
# Max, min, avg txBytes
# Max, min, avg rxBytes
# Max, min, avg duration

In [ ]:
# raw data

{
  "subscriberId":(11111111111,99999999999)
  "srcIP": "ipv4"
  "dstIP": "subnet"
  "srcPort": (1000,5000)
  "dstPort": (1000,5000)
  "txBytes": (0,1000000000)
  "rxBytes": (0,1000000000)
  "tcpFlag": (0,65)
  "startTime": "timestamp"
  "endTime": "timestamp"
  "protocolName": "tcp|udp|http"
  "protocolNumber": (0,255)
  "geoCountry": "country"
  "geoCity": "city"
  "latitude": (0,90)
  "longitude": (0,180)
}

# transformed data
{
  "transaction_time": "2019-10-27 23:22:17.848000",
  "subscriber_id": "100",
  "dst_subnet": "12.0.1.2/22",
  "number_of_unique_ips": "1",
  "number_of_unique_ports": "1",
  "number_of_records": "2",
  "max_tx_bytes": "15",
  "min_tx_bytes": "10",
  "avg_tx_bytes": "12.5",
  "max_rx_bytes": "40",
  "min_rx_bytes": "40",
  "avg_rx_bytes": "40.0",
  "max_duration": "100",
  "min_duration": "9",
  "avg_duration": "54.5"
}

# model features
{  "number_of_records": "2",
  "max_tx_bytes": "15",
  "min_tx_bytes": "10",
  "avg_tx_bytes": "12.5",
  "max_rx_bytes": "40",
  "min_rx_bytes": "40",
  "avg_rx_bytes": "40.0",
  "max_duration": "100",
  "min_duration": "9",
  "avg_duration": "54.5"
  }

In [4]:
from faker import Faker

In [5]:
faker = Faker()

In [ ]:
# python3 pipeline.py --streaming --project "electric-armor-395015" --topic "projects/electric-armor-395015/topics/test" --qps 10 --runner "DataflowRunner" --region europe-west2 temp_location gs://my_terraform_state_bucket/temp staging_location gs://my_terraform_state_bucket/staging

In [ ]:
# gcloud pubsub subscriptions pull test-sub --auto-ack --limit 1

In [1]:
from faker import Faker

In [3]:
client = Faker()

In [5]:
import random
from faker import Faker
from typing import NamedTuple
from datetime import datetime, timedelta

client = Faker()

class Network(NamedTuple):
    ipv4: str
    port: int
    protocol_name: str
    protocol_num: int
    
class User(NamedTuple):
    subscriber_id: str
    ipv4: str
    port: int

class NetworkPool:
    def __init__(self, num_dest_ip=10000):
        self._client = client
        self._num_dest_ip = num_dest_ip
        self._protocols = [("TCP", 6), ("UDP", 17), ("HTTP", 80), ("HTTPS", 443)]
        self._ip = self._generate_ipv4()
        
    def _ipv4(self):
        return random.choice(self._ip)
     
    def _port(self):
        return self._client.port_number("system")
    
    def _protocol_info(self):
        return random.choice(self._protocols)
    
    def _generate_ipv4(self):
        return [self._client.ipv4_private() for _ in range(self._num_dest_ip)]
    
    def get_network(self):
        protocol = self._protocol_info()
        return Network(self._ipv4(), self._port(), protocol[0], protocol[1])

class UserObject:
    def __init__(self):
        self.client = client
        
    def _subscriber_id(self):
        return self.client.uuid4()
    
    def _ipv4(self):
        return self.client.ipv4()
    
    def _port(self):
        return self.client.port_number("user")
    
    def get_user(self):
        return User(self._subscriber_id(), self._ipv4(), self._port())

class JsonEvent:
    max_request_bytes = 5000
    allowed_lag_sec = 10
    
    @classmethod
    def generate(cls, user, network):
        start_time = datetime.now()
        time_diff = random.uniform(0, cls.allowed_lag_sec)
        end_time = start_time + timedelta(seconds=time_diff)
        return {"subscriber_id": user.subscriber_id,
                "src_ip": user.ipv4,
                "src_port": user.port,
                "dest_ip": network.ipv4,
                "dst_port": network.port,
                "tx_bytes": cls._normalized_bytes(time_diff),
                "rx_bytes": cls._normalized_bytes(time_diff),
                "start_time": start_time.isoformat(),
                "end_time": end_time.isoformat(),
                "protocol_name": network.protocol_name,
                "protocol_num": network.protocol_num}
        
    @classmethod
    def _normalized_bytes(cls, lag_time):
        random_byte = random.uniform(10, 100)
        normalized_byte = random_byte * lag_time
        return int(min(normalized_byte, cls.max_request_bytes))
    

In [6]:
b = NetworkPool()

In [15]:
b.get_network()

Network(ipv4='192.168.250.238', port=586, protocol_name='UDP', protocol_num=17)